In [5]:
import cv2
import numpy as np
import os

In [6]:
###KNN

def distance(x,y):
    return np.sqrt(((x-y)**2).sum())

def knn(train,test,k=5):
    dist = []
    for i in range(train.shape[0]):
        #Get vector and label
        ix = train[i, :-1]
        iy = train[i,-1]
        
        d = distance(test,ix)
        dist.append([d,iy])
    #Sort on basis of distance
    nk = sorted(dist,key=lambda x:x[0])[:k]
    #Retrieve labels
    labels = np.array(nk)[:,-1]
    
    #Get frequency of each label
    pred = np.unique(labels,return_counts=True)
    index = np.argmax(pred[1])
    return pred[0][index]
###################



cap=cv2.VideoCapture(0)
face_cascade=cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip=0
dataset_path='./Data/'
face_data=[]
label=[]
class_id=0 #labels 
names={} # Dic for labels
for fx in os.listdir(dataset_path):
    if fx.endswith('.npy'):
        names[class_id]=fx[:-4]
        data_item=np.load(dataset_path+fx)
        face_data.append(data_item)
        target=class_id*np.ones((data_item.shape[0],))
        class_id+=1
        label.append(target)
face_dataset=np.concatenate(face_data,axis=0)
face_labels=np.concatenate(label,axis=0).reshape((-1,1))
trainset=np.concatenate((face_dataset,face_labels),axis=1)

while(True):
    ret,frame=cap.read()
    
    if ret==False:
        continue
    
    faces=face_cascade.detectMultiScale(frame,1.3,5)
    for face in faces:
        x,y,w,h=face
        offset=10
        face_section=frame[y-offset:y+h+offset,x-offset:x+w+offset]
        face_section=cv2.resize(face_section,(100,100),cv2.INTER_AREA)
        out=knn(trainset,face_section.flatten())
        pred_name=names[int(out)]
        cv2.putText(frame,pred_name,(x-10,y-10),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2,cv2.LINE_AA)
        cv2.rectangle(frame,(x,y),(x+w,y+h),(100,150,50),2)
    cv2.imshow("Faces",frame)
    key=cv2.waitKey(1) & 0xFF
    if key==ord('f'):
        break
cap.release()
cv2.destroyAllWindows()